Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.

In [1]:
from __future__ import print_function
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [ ]:
epochs = 200

callbacks = [EarlyStopping(monitor='val_loss', patience=10, verbose=1),
             ModelCheckpoint(filepath='weights.h5', verbose=1, save_best_only=True)]

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=callbacks)

model.save_weights('weights.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/200
60000/60000 [==============================] - 19s - loss: 0.3313 - acc: 0.8990 - val_loss: 0.0767 - val_acc: 0.9762
Epoch 2/200
60000/60000 [==============================] - 17s - loss: 0.1141 - acc: 0.9660 - val_loss: 0.0520 - val_acc: 0.9829
Epoch 3/200
60000/60000 [==============================] - 17s - loss: 0.0868 - acc: 0.9742 - val_loss: 0.0458 - val_acc: 0.9853
Epoch 4/200
60000/60000 [==============================] - 17s - loss: 0.0735 - acc: 0.9782 - val_loss: 0.0365 - val_acc: 0.9878
Epoch 5/200
60000/60000 [==============================] - 17s - loss: 0.0631 - acc: 0.9810 - val_loss: 0.0355 - val_acc: 0.9877
Epoch 6/200
60000/60000 [==============================] - 17s - loss: 0.0569 - acc: 0.9831 - val_loss: 0.0332 - val_acc: 0.9872
Epoch 7/200
60000/60000 [==============================] - 17s - loss: 0.0523 - acc: 0.9844 - val_loss: 0.0344 - val_acc: 0.9889
Epoch 8/200
20736/60000 [=========>............

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])